# Merged Tile AGB Zonal Stats Colation

This notebook looks for zonal stats subdirectories and the zonal stats csv outputs within them and concatenates all files into one data frame which is exported as a csv to the output directory.

The following conditions apply:

 - run after Seasonal Biomass Zonal Pipeline.
 - env = base

#### Import modules

In [1]:
import pandas as pd
from glob import glob
import os
from calendar import monthrange
from datetime import datetime

In [ ]:
drive = "G"
# field data date - date the data was processed into basal 
date = "20230407"
# sub dir for the tile zonal stat outputs
zonal_out_date = "20230407"

In [ ]:
# # was working
# drive = "D"
# date = "20230202"
# zonal_out_date = "202301"

In [ ]:
# Dictionary identifies the data structure of the reference image

dict_ = {
"dbg": "single",
"dp1": "seasonal",
"dp0": "single"
}

#### Define inputs and output locations

In [ ]:
# orig was working
# basal = r"{0}:\cdu\data\output\{1}\slats_tern_biomass.csv".format(drive, date)

In [ ]:
basal = r"{0}:\cdu\data\output\{1}\slats_tern_blue_biolib_biomass.csv".format(drive, date)
#basal = r"{0}:\cdu\data\output\{1}\slats_tern_blue_biolib_biomass_test.csv".format(drive, date)
print(basal)

In [ ]:
dir_ = r"{0}:\cdu\data\zonal_stats\tile\{1}".format(drive, zonal_out_date)
output_dir = r"{0}:\cdu\data\zonal_stats\output\{1}".format(drive, date)

In [ ]:
dir_

In [ ]:
output_dir

In [ ]:
def start_seasonal_date(date_):
    """ extract the end dates of the seasonal image zonal stats."""
    
    year = date_[:4]
    month = date_[4:]
    
    start_date = str(year) + str(month) + "01"
    
    return start_date
    

def end_seasonal_date(date_):
    
    """ extract the start dates of the seasonal image zonal stats."""
    #print("date: ", date_)
    year = str(date_[:4])
    month = str(date_[4:])
    #print("month: ", month)
    
    month_, day_range = monthrange(int(year), int(month))
    end_date = str(year) + str(month) + str(day_range)
    #print(end_date)
    return end_date


def im_date_season(df):
    """Collate start date of image into im_date column"""
    
    st_date_list = []
    e_date_list = []
    for i in df.im_name:
        #print(i)
        list_name = i.split("_")
        date = list_name[-2]
        st_date = date[1:7]
        start_date = start_seasonal_date(st_date)
        st_date_list.append(start_date)
        
        e_date = date[7:] 
        end_date = end_seasonal_date(e_date)
        e_date_list.append(end_date)
        
    df["im_s_date"] = st_date_list
    df["im_e_date"] = e_date_list
    
    return df
        
    
def im_date_annual(df):
    """Collate start date of image into im_date column"""
    
    st_date_list = []
    e_date_list = []
    for i in df.im_name:
        #print(i)
        list_name = i.split("_")
        date = list_name[-2]
        st_date = str(date) + "01"
        start_date = start_seasonal_date(st_date)
        st_date_list.append(start_date)
        
        e_date = str(date) + "12"
        end_date = end_seasonal_date(e_date)
        e_date_list.append(end_date)
        
    df["im_s_date"] = st_date_list
    df["im_e_date"] = e_date_list
    
    return df

    
def convert_to_datetime(df, col_nm_s, col_nm_d):
    
    date_list = []
    for t in df[col_nm_s]:
        
        i = str(t).strip()
        print("i: ", i)
        datetime_object = datetime.strptime(str(i), '%Y%m%d')
        date_list.append(datetime_object)
        print(datetime_object)
        #df[col_nm_d] =  pd.to_datetime(df[col_nm_s], format='%Y%m%d.%f')
        #date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
    df[col_nm_d] = date_list
    return df        

## AGB Basal data

In [ ]:
basal_df = pd.read_csv(basal)

In [ ]:
basal_df

In [ ]:
basal_df = convert_to_datetime(basal_df, "date", "basal_dt")
basal_df.sort_values(by='basal_dt', inplace=True)

In [ ]:
file_list = []
single_list = []
seasonal_list = []
dbg_list = []
dp0_list = []
dp1_list = []

print(dir_)
sub_list = next(os.walk(dir_))[1]
for i in sub_list:
    #print("i: ", i)
    sub_list_path = os.path.join(dir_, i)

    sub_sub_list = next(os.walk(sub_list_path))[1]
    #print(sub_sub_list)
    for n in sub_sub_list:
        #print("n: ", n)
        sub_sub_list_path = os.path.join(sub_list_path, n)
        #print(sub_sub_list_path)

        for file_ in glob(os.path.join(sub_sub_list_path, "*zonal_stats.csv")):
            print(file_)
            
            df1 = pd.read_csv(file_)
            
            if "date" in df1.columns and "im_date" not in df1.columns:
                #print(df1.columns)
                df1.rename(columns = {"date": "im_date"}, inplace = True)
                
            elif "date" not in df1.columns and "im_date" not in df1.columns:
                if "year" in df1.columns and "month" in df1.columns and "day" in df1.columns:
                    
                    im_date_list = []
                    for index, row in df1.iterrows():
                        #print(row["year"])
                        im_date_list.append(str(row["year"]) + str(row["month"]) + str(row["day"]))
                        
                    df1["im_date"] = im_date_list

            print(df1.columns)


            
            path_, f = os.path.split(file_)
            name_list = f.split("_")
            type_ = name_list[-3]
            print("type_", type_)
            
            data = dict_.get(type_)
            #print(data)
            
            if type_ == "dp1":
                print("Seasonal")
                print(df1.columns)
                df1 = convert_to_datetime(df1, "s_date", "image_s_dt")
                df1 = convert_to_datetime(df1, "e_date", "image_e_dt")
                # append to list
                
                dp1_list.append(df1)
                #print(df1.columns)

            elif type_ == "dp0":
                print("Single")
                #print(df1.columns)
                s_date_list = []
                e_date_list = []

                df1 = convert_to_datetime(df1, "im_date", "image_dt")
#                 df1 = convert_to_datetime(df1, "e_date", "image_e_dt")
                # append to list
                dp0_list.append(df1)
                #print(df1.columns)
                
            elif type_ == "dbg":
                print("DBG")
                print("+"*50)
                print(df1.columns)

                df1 = convert_to_datetime(df1, "im_date", "image_dt")
#                 df1 = convert_to_datetime(df1, "e_date", "image_e_dt")
                # append to list
                dbg_list.append(df1)
                print("appended to dbg_list")
                #print(df1.columns)
                

            else:

                # convert to date time
                df1 = convert_to_datetime(df1, "im_date", "image_dt")
                # append to list
                single_list.append(df1)


In [ ]:
dbg_list

In [ ]:
if len(dbg_list) > 1:
    #sort df on datetime data
    dbg = pd.concat(dbg_list, axis=0)
    
    dbg.sort_values(by="image_dt", inplace=True)
    print(dbg.shape)
    dbg.dropna(subset = ['b1_dbg_min'], inplace=True)
    print(dbg.info())
    #merge data with basal datset based on the nearest date to the field data colection
    dbg = pd.merge_asof(basal_df, dbg, left_on="basal_dt", right_on= "image_dt", by="site", direction="nearest")
        
    output_path = os.path.join(output_dir, "merged_slats_field_agb_dbg_tile.csv") #.format(sub_dir))
    dbg.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)
       

In [ ]:
if len(dp0_list) > 1:
    #sort df on datetime data
    dp0 = pd.concat(dp0_list, axis=0)
    dp0.sort_values(by="image_dt", inplace=True)
    dp0.dropna(subset = ['b1_dp0_min'], inplace=True)

    #merge data with basal datset based on the nearest date to the field data colection
    dp0 = pd.merge_asof(basal_df, dp0, left_on="basal_dt", right_on= "image_dt", by="site", direction="nearest")
        
    output_path = os.path.join(output_dir, "merged_slats_field_agb_dp0_tile.csv") #.format(sub_dir))
    dp0.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)

In [ ]:
if len(dp1_list) > 1:
    dp1 = pd.concat(dp1_list, axis=0)
    dp1_start = dp1.copy()
    dp1_end = dp1.copy()
    #sort df on datetime data
    dp1_start.sort_values(by="image_s_dt", inplace=True)
    dp1_start.dropna(subset = ['b1_dp1_min'], inplace=True)
    
    dp1_end.sort_values(by="image_e_dt", inplace=True)
    dp1_start.dropna(subset = ['b1_dp1_min'], inplace=True)

    dp1_s = pd.merge_asof(basal_df, dp1_start, left_on="basal_dt", right_on= "image_s_dt", by="site", direction='forward')
    dp1_e = pd.merge_asof(basal_df, dp1_end, left_on="basal_dt", right_on= "image_e_dt", by="site", direction="backward")

    # export concatinated dp1 zonalstats
    output_path = os.path.join(output_dir, "dp1_zonal_concat.csv")
    dp1_start.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)
    
    
    output_path = os.path.join(output_dir, "merged_slats_field_agb_dp1_start.csv")
    dp1_s.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)

    output_path = os.path.join(output_dir, "merged_slats_field_agb_dp1_end.csv")
    dp1_e.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)  

Could finish script here

## Annual and Dry season values

In [ ]:
if len(dp1_list) > 1:
    dp1 = pd.concat(dp1_list, axis=0)
    dp1_annual = dp1.copy()
    dp1_dry = dp1.copy()
    
    dp1_dry_df = dp1_dry[dp1_dry["s_month"] == 5]
    dp1_annual_df = dp1_annual[dp1_annual["s_month"] == 1]
    #print(test)
    #print(dp1_dry)
    #sort df on datetime data
    dp1_dry_df.sort_values(by="image_s_dt", inplace=True)
    dp1_dry_df.dropna(subset = ['b1_dp1_min'], inplace=True)
    
    dp1_annual_df.sort_values(by="image_s_dt", inplace=True)
    dp1_annual_df.dropna(subset = ['b1_dp1_min'], inplace=True)

    dp1_dry = pd.merge_asof(basal_df, dp1_dry_df, left_on="basal_dt", right_on= "image_s_dt", by="site", direction='nearest')
    dp1_annual = pd.merge_asof(basal_df, dp1_annual_df, left_on="basal_dt", right_on= "image_s_dt", by="site", direction="nearest")

    output_path = os.path.join(output_dir, "merged_slats_field_agb_dp1_dry.csv")
    dp1_dry.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)

    output_path = os.path.join(output_dir, "merged_slats_field_agb_dp1_annual.csv")
    dp1_annual.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path) 

In [ ]:
# for i in dp1_s.columns:
#     print(i)

## Merge surface reflectance (dbg) with single date FCv3 (dp0)

In [ ]:
dbg_dp0_df = pd.merge(dbg, dp0,  how='left', left_on=['site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha',
                  'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha',
                  'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt'],
                  right_on = ['site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha',
                  'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha',
                  'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt'])

In [ ]:
output_path = os.path.join(output_dir, "merged_slats_field_agb_single_date_dbg_dp0.csv")
dbg_dp0_df.to_csv(os.path.join(output_path), index=False)
print("File output to: ", output_path) 

In [ ]:
dbg_dp0_dp1s = pd.merge(dbg_dp0_df, dp1_s,  how='left', left_on=['site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha',
                  'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha',
                  'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt'],
                  right_on = ['site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha',
                  'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha',
                  'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt'])

In [ ]:
dict_col = {"uid_x_x": "uid", "image_x": "dbg_im", "year_x": "dbg_year",
           "month_x": "dbg_month", "day_x": "dbg_day", "im_date_x": "dbg_im_d", "image_dt_x": "dbg_im_dt",            
            "image_y": "dp0_im", "year_y": "dp0_year",
            "month_y": "dp0_month", "day_y": "dp0_day", "im_date_y": "dp0_date", "image_dt_y": "dp0_date_dt", "image": "dp1_im", "s_day": "dp1_s_day",
            "s_month": "dp1_s_month", "s_year": "dp1_s_year", "s_date": "dp1_s_date", "e_day": "dp1_e_day",
            "e_month": "dp1_e_month", "e_year": "dp1_e_year", "e_date": "dp1_e_date", "image_s_dt": "dp1_im_s_dt",
            "image_e_dt": "dp1_im_e_dt"}

In [ ]:
dbg_dp0_dp1s_col = dbg_dp0_dp1s.rename(dict_col, axis=1)

In [ ]:
print(list(dbg_dp0_dp1s_col))

In [ ]:
# for i in df_final.columns:
#     print(i)

In [ ]:
# Remove column name 'A'
dbg_dp0_dp1s_col.drop(["uid_y_x", "uid_x_y", "uid_y_y", "uid_x", "uid_y"], axis=1, inplace=True)

In [ ]:
# output_path = os.path.join(output_dir, "merged_slats_field_agb_tile_concat.csv")
# dbg_dp0_dp1s_col.to_csv(os.path.join(output_path), index=False)
# print("File output to: ", output_path) 

In [ ]:
output_path = os.path.join(output_dir, "merged_slats_field_agb_single_date_dbg_dp0_dp1s.csv")
dbg_dp0_dp1s.to_csv(os.path.join(output_path), index=False)
print("File output to: ", output_path) 


In [ ]:
if len(dp1_list) > 1:
    dp1 = pd.concat(dp1_list, axis=0)
    dp1_annual = dp1.copy()
    dp1_dry = dp1.copy()
    
    dp1_dry_df = dp1_dry[dp1_dry["s_month"] == 5]
    dp1_annual_df = dp1_annual[dp1_annual["s_month"] == 1]
    #print(test)
    #print(dp1_dry)
    #sort df on datetime data
    dp1_dry_df.sort_values(by="image_s_dt", inplace=True)
    dp1_dry_df.dropna(subset = ['b1_dp1_min'], inplace=True)
    
    dp1_annual_df.sort_values(by="image_s_dt", inplace=True)
    dp1_annual_df.dropna(subset = ['b1_dp1_min'], inplace=True)

    dp1_dry = pd.merge_asof(basal_df, dp1_dry_df, left_on="basal_dt", right_on= "image_s_dt", by="site", direction='nearest')
    dp1_annual = pd.merge_asof(basal_df, dp1_annual_df, left_on="basal_dt", right_on= "image_s_dt", by="site", direction="nearest")

    output_path = os.path.join(output_dir, "merged_slats_field_agb_dp1_dry.csv")
    dp1_dry.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path)

    output_path = os.path.join(output_dir, "merged_slats_field_agb_dp1_annual.csv")
    dp1_annual.to_csv(os.path.join(output_path), index=False)
    print("File output to: ", output_path) 

In [ ]:
dbg_dp0_df

#### Add DP1 Dry

In [ ]:
dbg_dp0_df_dp1_dry_df = pd.merge(dbg_dp0_df, dp1_dry,  how='left', left_on=['site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha',
                  'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha',
                  'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt'],
                  right_on = ['site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha',
                  'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha',
                  'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt'])

#### Add DP1 Annual

In [ ]:
dbg_dp0_df_dp1_dry_dp1_an_df = pd.merge(dbg_dp0_df_dp1_dry_df, dp1_annual,  how='left', left_on=['site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha',
                  'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha',
                  'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt'],
                  right_on = ['site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha',
                  'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha',
                  'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'basal_dt'])

In [ ]:
print(list(dbg_dp0_df_dp1_dry_dp1_an_df.columns))

In [ ]:
dict_ = {'uid_x_x': 'uid', 'uid_y_x', 'image_x', 'year_x', 'month_x', 'day_x',
        
        
        }



In [ ]:
list_col = ['uid', 'site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 
 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 
 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'geometry', 'basal_dt', 
 'uid_y_x', 'image_x', 'year_x', 'month_x', 'day_x', 'b1_dbg_count', 'b1_dbg_min', 
 'b1_dbg_max', 'b1_dbg_mean', 'b1_dbg_med', 'b1_dbg_std', 'b1_dbg_p25', 'b1_dbg_p50', 
 'b1_dbg_p75', 'b1_dbg_p95', 'b1_dbg_p99', 'b1_dbg_range', 'b2_dbg_count', 'b2_dbg_min', 
 'b2_dbg_max', 'b2_dbg_mean', 'b2_dbg_med', 'b2_dbg_std', 'b2_dbg_p25', 'b2_dbg_p50', 'b2_dbg_p75', 
 'b2_dbg_p95', 'b2_dbg_p99', 'b2_dbg_range', 'b3_dbg_count', 'b3_dbg_min', 'b3_dbg_max', 'b3_dbg_mean', 
 'b3_dbg_med', 'b3_dbg_std', 'b3_dbg_p25', 'b3_dbg_p50', 'b3_dbg_p75', 'b3_dbg_p95', 'b3_dbg_p99', 'b3_dbg_range', 
 'b4_dbg_count', 'b4_dbg_min', 'b4_dbg_max', 'b4_dbg_mean', 'b4_dbg_med', 'b4_dbg_std', 'b4_dbg_p25', 'b4_dbg_p50', 
 'b4_dbg_p75', 'b4_dbg_p95', 'b4_dbg_p99', 'b4_dbg_range', 'b5_dbg_count', 'b5_dbg_min', 'b5_dbg_max', 'b5_dbg_mean', 
 'b5_dbg_med', 'b5_dbg_std', 'b5_dbg_p25', 'b5_dbg_p50', 'b5_dbg_p75', 'b5_dbg_p95', 'b5_dbg_p99', 'b5_dbg_range', 
 'b6_dbg_count', 'b6_dbg_min', 'b6_dbg_max', 'b6_dbg_mean', 'b6_dbg_med', 'b6_dbg_std', 'b6_dbg_p25', 'b6_dbg_p50', 
 'b6_dbg_p75', 'b6_dbg_p95', 'b6_dbg_p99', 'b6_dbg_range', 
 'dbg_im_date', 'db_image_dt', 
 'uid_x_y', 'geometry_', 'uid_y_y', 'dp0_image', 'year_y', 'month_y', 'day_y', 
 'b1_dp0_min', 'b1_dp0_max', 'b1_dp0_mean', 'b1_dp0_count', 'b1_dp0_std', 
 'b1_dp0_med', 'b1_dp0_p25', 'b1_dp0_p50', 'b1_dp0_p75', 'b1_dp0_p95', 'b1_dp0_p99', 'b1_dp0_range', 
 'b2_dp0_min', 'b2_dp0_max', 'b2_dp0_mean', 'b2_dp0_count', 'b2_dp0_std', 'b2_dp0_med', 'b2_dp0_p25', 
 'b2_dp0_p50', 'b2_dp0_p75', 'b2_dp0_p95', 'b2_dp0_p99', 'b2_dp0_range', 'b3_dp0_min', 'b3_dp0_max', 
 'b3_dp0_mean', 'b3_dp0_count', 'b3_dp0_med', 'b3_dp0_p25', 'b3_dp0_p50', 'b3_dp0_p75', 'b3_dp0_p95', 
 'b3_dp0_p99', 'b3_dp0_range', 'b3_dp0_std', 
 'dp0_im_date', 'dp0_image_dt', 
 'uid_x_x', 'geometry_', 'uid_y_x', 'dp1d_image', 's_day_x', 's_month_x', 's_year_x', 's_date_x', 'e_day_x', 'e_month_x', 'e_year_x', 
 'dp1d_date', 'b1_dp1d_min', 'b1_dp1d_max', 'b1_dp1d_mean', 'b1_dp1d_count', 'b1_dp1d_std', 
 'b1_dp1d_med', 'b1_dp1d_p25', 'b1_dp1d_p50', 'b1_dp1d_p75', 'b1_dp1d_p95', 'b1_dp1d_p99', 
 'b1_dp1d_range', 'b2_dp1d_min', 'b2_dp1d_max', 'b2_dp1d_mean', 'b2_dp1d_count', 'b2_dp1d_std',
 'b2_dp1d_med', 'b2_dp1d_p25', 'b2_dp1d_p50', 'b2_dp1d_p75', 'b2_dp1d_p95', 'b2_dp1d_p99d', 'b2_dp1d_range',
 'b3_dp1d_min', 'b3_dp1d_max', 'b3_dp1d_mean', 'b3_dp1d_count', 'b3_dp1d_med', 'b3_dp1d_p25',
 'b3_dp1d_p50', 'b3_dp1d_p75', 'b3_dp1d_p95', 'b3_dp1d_p99', 'b3_dp1d_range', 'b3_dp1d_std', 'dp1d_im_s_dt', 
 'dp1d_im_e_dt', 'uid_x_y', 'geometry_', 'uid_y_y', 'dp1a_im', 's_day_y', 's_month_y', 's_year_y', 's_date_y', 'e_day_y', 'e_month_y', 
 'e_year_y', 'e_date_y', 'b1_dp1a_min', 'b1_dp1a_max', 'b1_dp1a_mean', 'b1_dp1a_count', 'b1_dp1a_std', 'b1_dp1a_med',
 'b1_dp1a_p25', 'b1_dp1a_p50', 'b1_dp1a_p75', 'b1_dp1a_p95', 'b1_dp1a_p99', 'b1_dp1a_range', 'b2_dp1a_min', 'b2_dp1a_max',
 'b2_dp1a_mean', 'b2_dp1a_count', 'b2_dp1a_std', 'b2_dp1a_med', 'b2_dp1a_p25', 'b2_dp1a_p50', 'b2_dp1a_p75', 
 'b2_dp1a_p95', 'b2_dp1a_p99', 'b2_dp1a_range', 'b3_dp1a_min', 'b3_dp1a_max', 'b3_dp1a_mean', 'b3_dp1a_count',
 'b3_dp1a_med', 'b3_dp1a_p25', 'b3_dp1a_p50', 'b3_dp1a_p75', 'b3_dp1a_p95', 'b3_dp1a_p99', 'b3_dp1a_range', 'b3_dp1a_std',
 'dp1a_im_s_dt', 'dp1a_im_e_dt']

In [ ]:
#df_si.rename(columns= list_)
dbg_dp0_df_dp1_dry_dp1_an_df.columns = list_col

In [ ]:
drop_list = ['uid_y_x', 'image_x', 'year_x', 'month_x', 'day_x', 'uid_x_y', 'uid_y_y','year_y', 'month_y', 'day_y',
'uid_x_x',  's_day_x', 's_month_x', 's_year_x', 's_date_x', 'e_day_x', 'e_month_x', 'e_year_x',
's_day_y', 's_month_y', 's_year_y', 's_date_y', 'e_day_y', 'e_month_y', 'e_year_y', 'e_date_y', 'geometry_']

In [ ]:
# Drop multiple columns
dbg_dp0_df_dp1_dry_dp1_an_df.drop(drop_list, axis = 1, inplace=True)

In [ ]:
output_path = os.path.join(output_dir, "dbg_dp0_df_dp1_dry_dp1_annual.csv")
dbg_dp0_df_dp1_dry_dp1_an_df.to_csv(os.path.join(output_path), index=False)
print("File output to: ", output_path)

In [ ]:
print(list(dbg_dp0_df_dp1_dry_dp1_an_df))

In [ ]:
dp0_an_df = dbg_dp0_df_dp1_dry_dp1_an_df[['site', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 
'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 
'c_agb_kg1ha',  'dp0_image', 'b1_dp0_min', 'b1_dp0_max', 'b1_dp0_mean', 'b1_dp0_std', 'b1_dp0_med', 'b1_dp0_p25', 
'b1_dp0_p50', 'b1_dp0_p75', 'b1_dp0_p95', 'b1_dp0_p99', 'b2_dp0_min', 'b2_dp0_max', 'b2_dp0_mean', 'b2_dp0_std', 
'b2_dp0_med', 'b2_dp0_p25', 'b2_dp0_p50', 'b2_dp0_p75', 'b2_dp0_p95', 'b2_dp0_p99', 'b3_dp0_min', 'b3_dp0_max', 
'b3_dp0_mean', 'b3_dp0_med', 'b3_dp0_p25', 'b3_dp0_p50', 'b3_dp0_p75', 'b3_dp0_p95', 'b3_dp0_p99', 'b3_dp0_std', 
'dp0_im_date']]

In [ ]:
output_path = os.path.join(output_dir, "dp0df_annual_clean.csv")
dp0_an_df.to_csv(os.path.join(output_path), index=False)
print("File output to: ", output_path)

In [ ]:
drop_list = [ 'date', 'lon_gda94', 'lat_gda94', 'dp1a_im_e_dt', 'geometry', 'uid', 'basal_dt', 'db_image_dt', 'dp0_image_dt',
             'dp1d_im_s_dt', 'dp1d_im_e_dt', 'dp1a_im_s_dt', 'dp1a_im_e_dt',
             'b1_dbg_count', 'b2_dbg_count', 'b3_dbg_count', 'b4_dbg_count', 'b5_dbg_count', 'b6_dbg_count', 'b1_dp0_count',
            'b2_dp0_count', 'b3_dp0_count','b1_dp1a_count', 'b2_dp1a_count', 'b3_dp1a_count',
            'b1_dbg_range', 'b2_dbg_range', 'b3_dbg_range', 'b4_dbg_range', 'b5_dbg_range', 'b6_dbg_range',
            'b1_dp0_range', 'b2_dp0_range', 'b3_dp0_range', 'b1_dp1a_range', 'b2_dp1a_range', 'b3_dp1a_range']

In [ ]:
# Drop multiple columns
dbg_dp0_df_dp1_dry_dp1_an_df.drop(drop_list, axis = 1, inplace=True)

In [ ]:
output_path = os.path.join(output_dir, "dbg_dp0_df_dp1_dry_dp1_annual_clean.csv")
dbg_dp0_df_dp1_dry_dp1_an_df.to_csv(os.path.join(output_path), index=False)
print("File output to: ", output_path)

# Export seasonal dp0 data to concat with NT mosaic

In [ ]:
print(list(dbg_dp0_df_dp1_dry_dp1_an_df.columns))

In [ ]:
# dp0_an_df = dbg_dp0_df_dp1_dry_dp1_an_df[['site', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 
# 'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 
# 'c_agb_kg1ha',  'dp0_image', 'b1_dp0_min', 'b1_dp0_max', 'b1_dp0_mean', 'b1_dp0_std', 'b1_dp0_med', 'b1_dp0_p25', 
# 'b1_dp0_p50', 'b1_dp0_p75', 'b1_dp0_p95', 'b1_dp0_p99', 'b2_dp0_min', 'b2_dp0_max', 'b2_dp0_mean', 'b2_dp0_std', 
# 'b2_dp0_med', 'b2_dp0_p25', 'b2_dp0_p50', 'b2_dp0_p75', 'b2_dp0_p95', 'b2_dp0_p99', 'b3_dp0_min', 'b3_dp0_max', 
# 'b3_dp0_mean', 'b3_dp0_med', 'b3_dp0_p25', 'b3_dp0_p50', 'b3_dp0_p75', 'b3_dp0_p95', 'b3_dp0_p99', 'b3_dp0_std', 
# 'dp0_im_date']]

In [2]:
lista =['uid_x_x', 'site', 'site_name_x', 'date', 'lon_gda94', 'lat_gda94', 'bio_l_kg1ha', 'bio_t_kg1ha', 'bio_b_kg1ha', 'bio_w_kg1ha', 'bio_br_kg1ha', 'bio_s_kg1ha', 'bio_r_kg1ha', 'bio_agb_kg1ha', 'c_l_kg1ha', 'c_t_kg1ha', 'c_b_kg1ha', 'c_w_kg1ha', 'c_br_kg1ha', 'c_s_kg1ha', 'c_r_kg1ha', 'c_agb_kg1ha', 'bio_bgb_kg1ha_x', 'bio_tb_kg1ha_x', 'basal_dt', 'uid_y_x', 'image_x', 's_day_x', 's_month_x', 's_year_x', 's_date_x', 'e_day_x', 'e_month_x', 'e_year_x', 'e_date_x', 'b1_dja_count', 'b1_dja_min', 'b1_dja_max', 'b1_dja_mean', 'b1_dja_med', 'b1_dja_std', 'b1_dja_p25', 'b1_dja_p50', 'b1_dja_p75', 'b1_dja_p95', 'b1_dja_p99', 'b1_dja_range', 'image_s_dt_x', 'uid_x_y', 'site_name_y', 'bio_bgb_kg1ha_y', 'bio_tb_kg1ha_y', 'uid_y_y', 'image_y', 's_day_y', 's_month_y', 's_year_y', 's_date_y', 'e_day_y', 'e_month_y', 'e_year_y', 'e_date_y', 'b1_dbi_count', 'b1_dbi_min', 'b1_dbi_max', 'b1_dbi_mean', 'b1_dbi_med', 'b1_dbi_std', 'b1_dbi_p25', 'b1_dbi_p50', 'b1_dbi_p75', 'b1_dbi_p95', 'b1_dbi_p99', 'b1_dbi_range', 'b2_dbi_count', 'b2_dbi_min', 'b2_dbi_max', 'b2_dbi_mean', 'b2_dbi_med', 'b2_dbi_std', 'b2_dbi_p25', 'b2_dbi_p50', 'b2_dbi_p75', 'b2_dbi_p95', 'b2_dbi_p99', 'b2_dbi_range', 'b3_dbi_count', 'b3_dbi_min', 'b3_dbi_max', 'b3_dbi_mean', 'b3_dbi_med', 'b3_dbi_std', 'b3_dbi_p25', 'b3_dbi_p50', 'b3_dbi_p75', 'b3_dbi_p95', 'b3_dbi_p99', 'b3_dbi_range', 'b4_dbi_count', 'b4_dbi_min', 'b4_dbi_max', 'b4_dbi_mean', 'b4_dbi_med', 'b4_dbi_std', 'b4_dbi_p25', 'b4_dbi_p50', 'b4_dbi_p75', 'b4_dbi_p95', 'b4_dbi_p99', 'b4_dbi_range', 'b5_dbi_count', 'b5_dbi_min', 'b5_dbi_max', 'b5_dbi_mean', 'b5_dbi_med', 'b5_dbi_std', 'b5_dbi_p25', 'b5_dbi_p50', 'b5_dbi_p75', 'b5_dbi_p95', 'b5_dbi_p99', 'b5_dbi_range', 'b6_dbi_count', 'b6_dbi_min', 'b6_dbi_max', 'b6_dbi_mean', 'b6_dbi_med', 'b6_dbi_std', 'b6_dbi_p25', 'b6_dbi_p50', 'b6_dbi_p75', 'b6_dbi_p95', 'b6_dbi_p99', 'b6_dbi_range', 'image_s_dt_y', 'uid_x_x', 'site_name_x', 'bio_bgb_kg1ha_x', 'bio_tb_kg1ha_x', 'uid_y_x', 'image_x', 's_day_x', 's_month_x', 's_year_x', 's_date_x', 'e_day_x', 'e_month_x', 'e_year_x', 'e_date_x', 'b1_dim_count', 'b1_dim_min', 'b1_dim_max', 'b1_dim_mean', 'b1_dim_med', 'b1_dim_std', 'b1_dim_p25', 'b1_dim_p50', 'b1_dim_p75', 'b1_dim_p95', 'b1_dim_p99', 'b1_dim_range', 'b2_dim_count', 'b2_dim_min', 'b2_dim_max', 'b2_dim_mean', 'b2_dim_med', 'b2_dim_std', 'b2_dim_p25', 'b2_dim_p50', 'b2_dim_p75', 'b2_dim_p95', 'b2_dim_p99', 'b2_dim_range', 'b3_dim_count', 'b3_dim_min', 'b3_dim_max', 'b3_dim_mean', 'b3_dim_med', 'b3_dim_std', 'b3_dim_p25', 'b3_dim_p50', 'b3_dim_p75', 'b3_dim_p95', 'b3_dim_p99', 'b3_dim_range', 'image_s_dt_x', 'uid_x_y', 'site_name_y', 'bio_bgb_kg1ha_y', 'bio_tb_kg1ha_y', 'uid_y_y', 'dis_image', 's_day_y', 's_month_y', 's_year_y', 's_date_y', 'e_day_y', 'e_month_y', 'e_year_y', 'e_date_y', 'band_x', 'dis_count', 'dis_min', 'dis_max', 'dis_mean', 'dis_sum', 'dis_std', 'dis_med', 'dis_major', 'dis_minor', 'dis_one', 'dis_two', 'dis_three', 'dis_four', 'dis_five', 'dis_six', 'dis_seven', 'dis_eight', 'dis_nine', 'dis_ten', 'image_s_dt_y', 'uid_x_x', 'site_name_x', 'bio_bgb_kg1ha_x', 'bio_tb_kg1ha_x', 'uid_y_x', 'dka_image', 'im_date', 'band_y', 'dka_count', 'dka_min', 'dka_max', 'dka_mean', 'dka_sum', 'dka_std', 'dka_med', 'dka_major', 'dka_minor', 'jan', 'feb', 'mar', 'april', 'may', 'june', 'july', 'aug', 'sep', 'oct', 'nov', 'dec', 's_date_x', 'e_date_x', 'image_s_dt_x', 'uid_x_y', 'site_name_y', 'bio_bgb_kg1ha_y', 'bio_tb_kg1ha_y', 'uid_y_y', 'stc_image', 's_day_x', 's_month_x', 's_year_x', 's_date_y', 'e_day_x', 'e_month_x', 'e_year_x', 'e_date_y', 'band', 'stc_count', 'stc_min', 'stc_max', 'stc_mean', 'stc_sum', 'stc_std', 'stc_med', 'stc_major', 'stc_minor', 'stc_one', 'stc_two', 'stc_three', 'stc_four', 'stc_five', 'stc_six', 'stc_seven', 'stc_eight', 'stc_nine', 'stc_ten', 'stc_elev', 'stc_twelv', 'stc_thirt', 'stc_fourt', 'stc_fift', 'stc_sixt', 'stc_sevent', 'image_s_dt_y', 'uid_x_x', 'site_name_x', 'bio_bgb_kg1ha_x', 'bio_tb_kg1ha_x', 'uid_y_x', 'image_y', 's_day_y', 's_month_y', 's_year_y', 's_date_x', 'e_day_y', 'e_month_y', 'e_year_y', 'e_date_x', 'b1_h99a2_count', 'b1_h99a2_min', 'b1_h99a2_max', 'b1_h99a2_mean', 'b1_h99a2_med', 'b1_h99a2_std', 'b1_h99a2_p25', 'b1_h99a2_p50', 'b1_h99a2_p75', 'b1_h99a2_p95', 'b1_h99a2_p99', 'b1_h99a2_range', 'image_s_dt_x', 'uid_x_y', 'site_name_y', 'bio_bgb_kg1ha_y', 'bio_tb_kg1ha_y', 'uid_y_y', 'image_x', 's_day_x', 's_month_x', 's_year_x', 's_date_y', 'e_day_x', 'e_month_x', 'e_year_x', 'e_date_y', 'b1_fpca2_count', 'b1_fpca2_min', 'b1_fpca2_max', 'b1_fpca2_mean', 'b1_fpca2_med', 'b1_fpca2_std', 'b1_fpca2_p25', 'b1_fpca2_p50', 'b1_fpca2_p75', 'b1_fpca2_p95', 'b1_fpca2_p99', 'b1_fpca2_range', 'image_s_dt_y', 'uid_x', 'site_name', 'bio_bgb_kg1ha', 'bio_tb_kg1ha', 'uid_y', 'image_y', 's_day_y', 's_month_y', 's_year_y', 's_date', 'e_day_y', 'e_month_y', 'e_year_y', 'e_date', 'b1_dp1_min', 'b1_dp1_max', 'b1_dp1_mean', 'b1_dp1_count', 'b1_dp1_std', 'b1_dp1_med', 'b1_dp1_p25', 'b1_dp1_p50', 'b1_dp1_p75', 'b1_dp1_p95', 'b1_dp1_p99', 'b1_dp1_range', 'b2_dp1_min', 'b2_dp1_max', 'b2_dp1_mean', 'b2_dp1_count', 'b2_dp1_std', 'b2_dp1_med', 'b2_dp1_p25', 'b2_dp1_p50', 'b2_dp1_p75', 'b2_dp1_p95', 'b2_dp1_p99', 'b2_dp1_range', 'b3_dp1_min', 'b3_dp1_max', 'b3_dp1_mean', 'b3_dp1_count', 'b3_dp1_med', 'b3_dp1_p25', 'b3_dp1_p50', 'b3_dp1_p75', 'b3_dp1_p95', 'b3_dp1_p99', 'b3_dp1_range', 'b3_dp1_std', 'image_s_dt', 'image_e_dt']
for i in lista:
    print("'" + i + "'")

'uid_x_x'
'site'
'site_name_x'
'date'
'lon_gda94'
'lat_gda94'
'bio_l_kg1ha'
'bio_t_kg1ha'
'bio_b_kg1ha'
'bio_w_kg1ha'
'bio_br_kg1ha'
'bio_s_kg1ha'
'bio_r_kg1ha'
'bio_agb_kg1ha'
'c_l_kg1ha'
'c_t_kg1ha'
'c_b_kg1ha'
'c_w_kg1ha'
'c_br_kg1ha'
'c_s_kg1ha'
'c_r_kg1ha'
'c_agb_kg1ha'
'bio_bgb_kg1ha_x'
'bio_tb_kg1ha_x'
'basal_dt'
'uid_y_x'
'image_x'
's_day_x'
's_month_x'
's_year_x'
's_date_x'
'e_day_x'
'e_month_x'
'e_year_x'
'e_date_x'
'b1_dja_count'
'b1_dja_min'
'b1_dja_max'
'b1_dja_mean'
'b1_dja_med'
'b1_dja_std'
'b1_dja_p25'
'b1_dja_p50'
'b1_dja_p75'
'b1_dja_p95'
'b1_dja_p99'
'b1_dja_range'
'image_s_dt_x'
'uid_x_y'
'site_name_y'
'bio_bgb_kg1ha_y'
'bio_tb_kg1ha_y'
'uid_y_y'
'image_y'
's_day_y'
's_month_y'
's_year_y'
's_date_y'
'e_day_y'
'e_month_y'
'e_year_y'
'e_date_y'
'b1_dbi_count'
'b1_dbi_min'
'b1_dbi_max'
'b1_dbi_mean'
'b1_dbi_med'
'b1_dbi_std'
'b1_dbi_p25'
'b1_dbi_p50'
'b1_dbi_p75'
'b1_dbi_p95'
'b1_dbi_p99'
'b1_dbi_range'
'b2_dbi_count'
'b2_dbi_min'
'b2_dbi_max'
'b2_dbi_mean'
'b2_dbi_med